In [1]:
!pip install pandas numpy faiss-cpu sentence-transformers openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 82.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
dataset = load_dataset("vishal-burman/c4-faqs")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002-afb6ac49c38b63(…):   0%|          | 0.00/151M [00:00<?, ?B/s]

data/train-00001-of-00002-c962114f6120a9(…):   0%|          | 0.00/150M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

In [3]:
# Check what keys/columns are available
print(dataset)
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['url', 'faq_pairs'],
        num_rows: 150000
    })
})
{'url': 'http://www.masters-degree-in-education.org/faq/do-you-need-a-masters-in-education-degree-to-qualify-for-tenure/', 'faq_pairs': [{'question': 'Do You Need a Master’s in Education Degree To Qualify for Tenure?', 'answer': 'If you are currently an educator or you are mapping out the route that you will take to become one, you may want to know if you need a Master in Education degree to qualify for tenure. To work as a full-time, tenure-track educator, you must meet certain requirements and prove that you are skilled and passionate about your role. Tenure offers you great protection in your career but every state and educational system has their own path that you must take. Read on, and find out when a master’s is and is not required.'}, {'question': 'What is a Tenure and Why is it So Important?', 'answer': 'Tenure is technically defined as the right to due process so that

In [8]:
import pandas as pd
faq_list = []
# dataset['train'] is a list of rows
for row in dataset['train']:
    # each row has a list of faq_pairs
    for faq in row['faq_pairs']:
        faq_list.append({
            'Question': faq['question'].lower().strip(),
            'Answer': faq['answer'].strip()
        })
# Convert to a pandas DataFrame
df = pd.DataFrame(faq_list)
# Inspect
df.head(), df.shape

(                                            Question  \
 0  do you need a master’s in education degree to ...   
 1       what is a tenure and why is it so important?   
 2  how long does it take to become a tenured educ...   
 3                  what should i bring for the tour?   
 4                                can kids join too ?   
 
                                               Answer  
 0  If you are currently an educator or you are ma...  
 1  Tenure is technically defined as the right to ...  
 2  Only about one-third of college professors are...  
 3  Nothing special, just be ready to walk for bit...  
 4  Yes, we accept kids from 6 years old and on! W...  ,
 (846722, 2))

In [9]:
df.to_csv("c4_faqs.csv", index=False)

In [10]:
#Generate Embeddings
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['Question'].tolist())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
#FAISS Index
import faiss
import numpy as np
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))

In [12]:
#Test Retrieval
def answer_query(query):
    q_vec = model.encode([query])
    D, I = index.search(np.array(q_vec), k=1)
    return df.iloc[I[0][0]]['Answer']
print(answer_query("how long do they take to fit?"))

This really depends upon the size & quantity of the shutters & what they're being fitted to (wooden window frames are quicker to fit to than plastered reveals), however most jobs are completed within half a day. Naturally, we can give you an accurate time frame before you place your order. 9.


In [13]:
# Step 1: Interactive query loop
while True:
    user_input = input("Ask a question (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        print("Exiting assistant. Goodbye!")
        break
    answer = answer_query(user_input)
    print("Answer:", answer)

Ask a question (or type 'exit' to quit): exit
Exiting assistant. Goodbye!
